## Agents with function calling


In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os
from openai import AzureOpenAI
from helpers import function_to_schema

client = AzureOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],    
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-05-01-preview",
)

Create agent

In [8]:
from helpers import function_to_schema

def get_weather(place: str, date: str):
    """
    Get weather at specific location on a particular date.
    """
    
    weather_condition = "wintry and cold"
    temperature = 3
    return f"""Here is some information about the weather in {place} on {date}:\n
        - The weather is: {weather_condition}.\n
        - The temperature is: {temperature} degrees Celsius.\n
    """
    
planning_agent = client.beta.assistants.create(
    name="Plan packing for a trip assistant",
    instructions=f"""You are a helpful AI assistant who helps people pack for a trip.
    You should be able to provide a list of items that a person should pack for a trip based on weather at their destination.
    The user must provide the name of their destination and the date of their trip. If they do not provide that, ask the user to provide the missing information. Always start such request with 'Please provide'.
    To get the weather details for a location, use the following function:
    - get_weather""",
    tools=[function_to_schema(get_weather)],
    model = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"), 
)

In [50]:
print(function_to_schema(get_weather))

{'type': 'function', 'function': {'name': 'get_weather', 'description': 'Get weather at specific location on a particular date.', 'parameters': {'type': 'object', 'properties': {'place': {'type': 'string'}, 'date': {'type': 'string'}}, 'required': ['place', 'date']}}}


In [5]:
import time
import json
from IPython.display import clear_output

def send_and_run(client, assistant_id, thread_id, content):
    client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=content
    )
    return client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id
    )
    
def wait_on_run(run):
    start_time = time.time()
    status = run.status
    while status not in ["completed", "cancelled", "expired", "failed", "requires_action"]:
        time.sleep(5)
        run = client.beta.threads.runs.retrieve(thread_id=run.thread_id,run_id=run.id)
        print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
        status = run.status
        print(f'Status: {status}')
        clear_output(wait=True)
    return run


def call_function(run):
    tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
    name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)
    print("Waiting for custom Function:", name)
    print("Function arguments:")
    print(arguments)
    tool_outputs = []
    match name:
      case "get_weather":
        tool_outputs.append({"tool_call_id": tool_call.id, "output": get_weather(arguments["place"], arguments["date"])})
      case _:
        print("Unknown function")
        # run = client.beta.threads.runs.update(
        #   thread_id=thread.id,
        #   run_id=run.id,
        #   outputs={"weather": weather}
        # )
    return client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        run_id=run.id,
        tool_outputs= tool_outputs,
)

In [ ]:
thread = client.beta.threads.create()
completed = False
msg = "I am going to visit Paris on Monday for a week."
agent = planning_agent
run = send_and_run(client, agent.id, thread.id, msg)
while True:
  run = wait_on_run(run)
  match run.status:
    case "completed":
      msg = client.beta.threads.messages.list(thread_id=thread.id, order="desc").data[0].content[0].text.value
      if msg.startswith("Please provide"):
        print(msg)
        user_input = input(f"{msg}:> ")
        run = send_and_run(client, agent.id, thread.id, user_input)
      else:
        print(msg)
        break
    case "requires_action":
      run = call_function(run)
    case _:
      print(f"Unexpected action: {run.status}")
      break
print("Done")

Return status: completed
Based on the weather forecast for Paris on November 31, 2025, which is wintry and cold with a temperature of 3 degrees Celsius, here is a list of items you should pack for your trip:

### Clothing:
1. Warm winter coat
2. Sweaters or thermal tops
3. Long-sleeve shirts
4. Warm pants (jeans or thermal leggings)
5. Scarf, gloves, and a warm hat
6. Warm socks
7. Comfortable walking shoes (waterproof if possible)
8. Undergarments

### Accessories:
9. Umbrella (in case of snow or rain)
10. Sunglasses (the winter sun can still be bright)
11. Backpack or day bag for daily outings

### Toiletries:
12. Moisturizer (the cold weather can be drying)
13. Lip balm
14. Regular toiletries (toothbrush, toothpaste, etc.)

### Electronics:
15. Phone and charger
16. Camera (optional)

Make sure to check the weather closer to your departure date in case anything changes! Enjoy your trip to Paris!
Done


In [7]:
print(client.beta.threads.messages.list(thread_id=thread.id, order="desc").data[0].content[0].text.value)

Could you please provide the exact date of your trip?
